In [1]:
import numpy as np
from random import randint
import time
import bisect
import math
import scipy
from scipy import signal
from copy import copy
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


%load_ext pycodestyle_magic

In [ ]:
points = np.zeros((1000,1))
middle = 500
points[middle] = 1
y0st = gaussian_filter(copy(points), 100, order=0)
y2st = gaussian_filter(copy(points), 100, order=2)

print(sum(y0st), sum(abs(y2st)))

y0st = y0st/ np.linalg.norm(y0st)
y2st = y2st/ np.linalg.norm(y2st)

print(sum(y0st), sum(abs(y2st)))

plt.title('Gaussian Distribution')
plt.plot(range(-500, 500 , 1,), y0st, label='Normalized')

plt.plot(range(-500, 500, 1), y2st, label='Normalized second order derivative')


plt.legend(loc='upper left', bbox_to_anchor=(1, 1),shadow=True, ncol=1)
plt.ylabel('y')
plt.xlabel('x')

plt.figure()
plt.plot(range(-500, 500 , 1,), -y0st, label='Normalized inverse')
plt.plot(range(-500, 500 , 1,), -y0st + y2st, label='Sum inverse and second order derivative')


plt.legend(loc='upper left', bbox_to_anchor=(1, 1),shadow=True, ncol=1)
plt.ylabel('y')
plt.xlabel('x')
plt.show()


In [ ]:
def xyzGaus(sigmas, location):
    c = np.divide(1, np.multiply(sigmas, math.sqrt(2 * math.pi)))    
    c2 = np.exp(-np.divide(np.multiply(location, location), np.multiply(2, np.power(sigmas, 2))))
    return sum(np.multiply(c, c2))
    
def xyzGausSecondOrder(sigmas, location):
    sigmasSquared = np.power(sigmas, 2)
    sigmasfifth = np.power(sigmas, 5)
    locationSquared = np.power(location, 2)
    c = - np.divide(np.subtract(sigmasSquared, locationSquared), np.multiply(sigmasfifth, math.sqrt(2 * math.pi)))
    c2 = np.exp( - (np.divide(locationSquared, np.multiply(2,sigmasSquared))))
    return sum(np.multiply(c,c2))

In [ ]:
###### print(xyzGaus(np.asarray([100,3,3]), 0,100,0))

ran = np.arange(-1000, 1000, 0.1)

y = []
y1 = []
for x in ran:
    y1.append(-xyzGaus(np.asarray([50]), np.asarray([x])))
    y.append(-xyzGausSecondOrder(np.asarray([25]), np.asarray([x])))
    
# y = y / np.linalg.norm(y)    
y = np.multiply(y,[25**math.e])
y1 = y1 / np.linalg.norm(y1)
plt.plot(ran, y)
plt.plot(ran, y1)
    

In [ ]:
def euclideanDist(a, b): 
    return np.linalg.norm(np.asarray(a)-np.asarray(b))

def withinBoundary(location, boundary):
    z, x, y = location
    zMax, xMax, yMax = boundary 
    if z < 0 or x < 0 or y < 0 or z >= zMax or x >= xMax or y >= yMax:
        return False
    return True

In [ ]:
def createField(grid, points=[[0,0,0]], sigma=3, value=1, order=0):
    points = [[int(num) for num in point] for point in points]
    
    cpGrid = copy(grid)
    for point in points:
        cpGrid[point[0]][point[1]][point[2]] = value
    
    gausedArray = scipy.ndimage.filters.gaussian_filter(cpGrid, sigma, order=order,mode='constant')
    
    return gausedArray/np.linalg.norm(gausedArray)

def showField(z, x, y, c, drones, center, mainDrone=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    img = ax.scatter(x, y, z, c=c, s=00000.1, cmap=plt.cm.RdYlGn.reversed())

    ax.set_xlim3d(0,1)
    ax.set_ylim3d(0,200)
    ax.set_zlim3d(0,200)

    fig.colorbar(img)

    for z,x,y in drones:
        if mainDrone is None:
            ax.scatter(x, y, z, c='b', s=20, marker='^', depthshade=False)
        else:
            if z == mainDrone[0] and x == mainDrone[1] and y == mainDrone[2]:
                ax.scatter(x, y, z, c='g', s=50, marker='^', depthshade=False)
            else:
                ax.scatter(x, y, z, c='b', s=20, marker='^', depthshade=False)
        

    z,x,y = center
    ax.scatter(x, y, z, c='r', s=20, marker='*', depthshade=False)


    ax.view_init(0, 0)
    plt.show()     
    
def gridToPlotValues(grid):
    (depth, row, column) = grid.shape
    coordinates = np.array([np.array([x, y, z, grid[z][x][y]])\
                for x in range(row)\
                for y in range(column) for z in range(depth) ])

    x = coordinates[:,0]
    y = coordinates[:,1]
    z = coordinates[:,2]
    c = coordinates[:,3]
    return z, x, y ,c

def combineFields(fields):
    combined = []
    for field in fields:
        if combined == []:
            combined = field
        else:
            combined = combined + field
    return combined / np.linalg.norm(combined)


In [ ]:
def gausAtDistance(sigma, location, order):
    if order == 0:
        return - xyzGaus(sigma, location)
    if order == 2:
        return - xyzGausSecondOrder(sigma, location)

class drone:
    def __init__(self):
        self.type = "drone"
        self.location = np.array([0,0,0])

    def setLocation(self, z, x, y):
        self.location = np.array([z,x,y])
        
    def force(self, grid, sigma, value, order):
        self.field = createField(grid, [self.location], sigma=sigma, value=value, order=order)
        return self.field
    
    def getBestStep(self, gravityVars, magnaticVars, boundary):
        steps = self.possibleSteps(boundary)

        center, sigmas = gravityVars
        gravity = [gausAtDistance(sigmas, center - step, 0) for step in steps]
        
        drones, sigmas = magnaticVars
        magnetic = [sum([gausAtDistance(sigmas, drone.location - step, 2) for drone in drones if not drone is self]) for step in steps]
        
#         print(gravity, magnetic)
        
        total = np.sum([gravity, magnetic], axis=0)
#         print(total, np.argmin(total))
        return steps[np.argmin(total)]
        
    
    def world(self, drones, magnaticField):
        droneForces = []
        for drone in drones:
            if not drone is self:
                droneForces.append(drone.field)
        
        gravity = combineFields(droneForces)
            
        return combineFields([magnaticField, gravity]) 
    
    def possibleSteps(self, boundary):
        (z, x, y) = self.location
        options = [[z, x, y]]
        for i in (-1, 1):
            options.append(np.array([z+i, x, y]))
            options.append(np.array([z, x+i, y]))
            options.append(np.array([z, x, y+i]))
        return np.array([location for location in options 
                    if withinBoundary(location, boundary)])
    
    def __str__(self):
        return str(id(self))


In [ ]:
class simulation:
    def __init__(self, numberOfDrones, depth, width, height):
        self.numberOfDrones = numberOfDrones
        self.dimensions = (depth, width, height)
        self.drones = []
        self.grid = grid(depth, width, height)
        self.createNDrones(numberOfDrones)
        
    def showFields(self):
        z, x, y, c = gridToPlotValues(self.grid.grid)
        showField(z, x, y, c, sim.droneLocations(), sim.droneCenter(), mainDrone=sim.droneLocations()[0])
    
    def showDrones(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        (depth, width, height) = self.dimensions
        
        ax.set_xlim3d(0,width)
        ax.set_ylim3d(0,height)
        ax.set_zlim3d(0,depth)

        for z,x,y in self.droneLocations():
            ax.scatter(x, y, z, c='b', s=20, marker='^', depthshade=True)
            
        z,x,y = self.droneCenter()
        ax.scatter(x, y, z, c='r', s=20, marker='*', depthshade=True)


        ax.view_init(40, 40)
        return fig             
      
    def getCoordinateNextMove(self, drone, grid):
        locations = self.getOptions(drone.location)
        
        optionsWithValue = []
        for location in locations:
            (z,x,y) = location
            optionsWithValue = optionsWithValue + [[z, x, y, grid[z][x][y]]]
        best = min(optionsWithValue, key=lambda x: x[3])
        drone.location = np.array([best[0], best[1], best[2]])
        
    def moveDrones(self):
        center = self.droneCenter()
        droneLocations = self.drones
        sigmaGravity = np.array(list(map(lambda x: x/2, self.dimensions)))
        sigmaMagnetic = np.array(list(map(lambda x: x/20, self.dimensions)))
        
        moves = []
        for drone in self.drones:
            moves.append([drone, drone.getBestStep([center, sigmaGravity], [droneLocations,  sigmaMagnetic], self.dimensions)])
            
        for move in moves:
            drone, step = move
            drone.location = step
        
        return True
    
    def createNDrones(self, n):
        drones = []
        for i in range(n):
            current = drone()
            drones.append(current)
            self.grid.placeDroneRandomCoordinate(current)
        self.drones = drones
        
    def droneLocations(self):
        return [drone.location for drone in self.drones]
        
    def layFields(self, grid):
        droneForces = []
        for drone in self.drones:
            droneForces.append(drone.force(grid, sigma=map(lambda x: x/20, self.dimensions), value=-1, order=2))
        gravity = combineFields(droneForces)
        
        center = self.droneCenter()
        magnaticField = createField(grid, [center], sigma=map(lambda x: x/2, self.dimensions), value=-2)
        self.grid.setGrid(combineFields([magnaticField, gravity]))

    def droneCenter(self):
        locations = []
        for current in self.drones:
            locations.append(current.location)
        return np.mean(np.array(locations), axis=0)
            
    def updateDroneGrids(self):
        for currentDrone in self.drones:
            currentDrone.droneLocationsPOV(self.drones)

In [ ]:
class grid:
    def __init__(self, depth, width, height, stepSize=1):
        self.dimensions = (depth, width, height)
        self.grid = np.zeros(self.dimensions, dtype='float32')
        self.empty = self.grid

    def placeOnGrid(self, value, z, x, y):
        self.grid[z, x, y] = value

    def setGrid(self, newGrid):
        self.grid = newGrid
        
    def getRandomCoordinate(self):
        (depth, width, height) = self.dimensions
        z = randint(0, depth-1)
        x = randint(0, width-1)
        y = randint(0, height-1)
        return (z, x, y)
    
    def getOnGrid(self, z, x, y):
        return self.grid[z, x, y]

    def placeDroneRandomCoordinate(self, drone):
        (z, x, y) = self.getRandomCoordinate()
#         self.placeOnGrid(drone, z, x, y)
        drone.setLocation(z,x,y)
        return (z, x, y)

    def showGrid(self):
        for layer in self.grid:
            print(np.asarray([[str(value) for value in row] for row in layer]))


In [ ]:
sim = simulation(5, 200, 200, 200)
sim.layFields(sim.grid.empty)
sim.showDrones()
path = "try3/"
for x in range(300):
    sim.moveDrones()
    sim.showDrones().savefig(path+str(x)+'.png', format="png")

# sim.show()
# sim.show()

In [ ]:

sim.showfield()
# plt.show()

In [ ]:
import os
def sortKeyFunc(s):
    return int(os.path.basename(s)[:-4])

In [ ]:
import cv2
import numpy as np
import glob


files = glob.glob('C:/Users/Mathieu/Documents/scriptie/try3/*.png')
files.sort(key=sortKeyFunc)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output3.avi',fourcc, 20.0, (432,288), 1)
for filename in files[0:400]:
    img = cv2.imread(filename)
#     [print(row) for row in img]
    cv2.imshow('frame',img)
    out.write(img)
    cv2.imwrite(filename, img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#     break
out.release()
cv2.destroyAllWindows()

In [ ]:
class droneExtra:
        def layPath(self, origin, goal, grid):
        newPath = pathFinder(origin, goal, grid.dimensions, self)
        return newPath.findPath()
    
    def droneLocationsPOV(self, drones):
        locations = []
        for current in drones:
            if current != self:
                locations.append(current.location - self.location)
        print(np.array(locations), np.mean(np.array(locations), axis=0))
        
        createGridThatFits(locations)
        

    def createGridThatFits(self, locations):
        largeNumber = 999999999
        zmin, zmax = largeNumber, -largeNumber
        xmin, xmax = largeNumber, -largeNumber
        ymin, ymax = largeNumber, -largeNumber    
        for z, x, y in locations:
            zmin = min(zmin, z)
            zmax = max(zmax, z)
            xmin = min(zmin, x)
            xmax = max(zmax, x)
            ymin = min(zmin, y)
            ymax = max(zmax, y)
        depth = abs(zmax - zmin)
        width = abs(xmax - xmin)
        heigth = abs(ymax - ymin)
        
        newGrid = grid(depth, width, height)
        
        
        
        
    def grid2OwnGrid(self, grid):
        pass
        

In [ ]:
class path:
    def __init__(self, drone, bestOption):
        self.drone = drone
        self.option2Path(bestOption)
        
    def option2Path(self, option):
        (current, cost, dist, value, history) = option
        history.append(current)
        self.path = history
        self.cost = cost
        self.origin = history[0]
        self.goal = current
        
    def __str__(self):
        return str(id(self))
    
class crossing:
    def __init__(self, origins, goals, times):
        self.origins = origins
        self.goals = goals
        self.times = times
    
    def __str__(self):
        return str(id(self))
        
    def addPath(self, path):
        self.origins.append(path.origin)
        self.origins.append(path.goal)
        self.origins.append(path.time)

In [ ]:
class pathFinder:
    def __init__(self, origin, goal, boundary, drone):
        self.goal = goal
        self.drone = drone
        self.location = origin
        self.cost = 0
        self.options = [(origin, 0, euclideanDist(origin, goal), euclideanDist(origin, goal), [])]
        self.boundary = boundary
        
    def __str__(self):
        return str(id(self))

    def getOptions(self, current, boundary, history):
        (z, x, y) = current
        options = []
        for i in (-1, 1):
            options.append((z+i, x, y))
            options.append((z, x+i, y))
            options.append((z, x, y+i))
        return [location for location in options 
                    if withinBoundary(location, boundary) 
                    and not location in history]

    def expandOption(self, option):
        (current, cost, dist, value, history) = option
        possibleMoves = self.getOptions(current, self.boundary, history)
        if self.goal in possibleMoves:
            possibleMoves = [self.goal]
        newHistory = history + [current]
        expandedOptions = self.addOptionData(possibleMoves, cost, newHistory)
        self.options += expandedOptions
        
    def addOptionData(self, moves, cost, history):
        expandedOptions = []
        newCost = cost + 1
        for move in moves:
            dist = euclideanDist(move, self.goal)
            value = newCost + dist
            expandedOptions.append((move, newCost, dist, value, history))
        return expandedOptions
    
    def sortOptions(self):
        self.options = sorted(self.options, key=lambda x: (x[3],-x[1]) ) # first sort by approx total cost than sort on most distance traveled
      
    def findPath(self):
        current = self.options.pop()
        while current[0] != self.goal :
            self.expandOption(current)
            self.sortOptions()
            arr = []
            for option in self.options:
                arr.append((option[3], option[1]) )
            current = self.options.pop(0)
        
        
        return path(self.drone, current)

In [ ]:
sim.updateDroneGrids()

In [ ]:
field = grid(10, 50, 50)

dronesLocation = []
for i in range(3):
    dronesLocation.append(field.placeDroneRandomCoordinate())

In [ ]:

go1 = (0,25,25)
go2 = (0,26,26)
go3 = (0,24,24)

z, x, y = dro1

drone1 = field.getOnGrid(z,x,y)

route = drone1.layPath(dro1, go1, field)

field.layPath(route)
# print(np.array([[euclideanDist(location, goal) for goal in goals] for location in dronesLocation]))



In [ ]:
route.path

In [ ]:
# (z, x, y) = field.getRandomCoordinate()
field.showGrid()

In [ ]:
def bench(N):
    randomVars = np.random.randint(1, 200, (N, 6))

    start = time.time()
    for sigz, sigx, sigy, z, x, y in randomVars:
        xyzGaus(np.asarray([sigz, sigx, sigy]), z, x, y)
    end = time.time()
    total = end - start    
    print(total)

    randomVars = np.random.randint(1, 200, (N, 6))
    # losgetrokken
    start = time.time()
    for sigz, sigx, sigy, z, x, y in randomVars:
        instantGaus(np.asarray([sigz, sigx, sigy]), z, x, y)
    end = time.time()
    total = end - start 
    print(total)
    
    return ans1, ans2
    
ans1, ans2 = bench(1000000)
np.array_equal(ans1,ans2)